In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import Callback
from keras import backend as K
from keras.utils.np_utils import to_categorical

In [ ]:
from bqplot import pyplot as plt
from bqplot import ColorScale, OrdinalScale
from ipywidgets import HBox, VBox, Layout, FloatProgress, Label, IntSlider, Dropdown
from IPython.display import display
import numpy as np

In [ ]:
batch_size = 128
num_classes = 10
epochs = 10

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
train_test_dd = Dropdown(options=['Train', 'Test'])
sel_slid = IntSlider(min=0, max=x_train.shape[0]-1, description='Index')
fig_map = plt.figure()
heat_map = plt.heatmap(color=x_train[0][::-1], scales={'color': ColorScale(scheme='Greys')},
                      axes_options={'x': {'visible': False},
                                    'y': {'visible': False},
                                    'color': {'visible': False}})

def slid_changed(change):
    heat_map.color = x_train[sel_slid.value][::-1]
    
def dd_changed(change):
    sel_slid.value = 0
    if train_test_dd.value == 'Train':
        sel_slid.max = x_train.shape[0]
    else:
        sel_slid.max = x_test.shape[0]
    
    slid_changed(None)
    
sel_slid.observe(slid_changed, 'value')
train_test_dd.observe(dd_changed, 'value')

In [ ]:
VBox([HBox([train_test_dd, sel_slid]), fig_map])

In [ ]:
class Dashboard(Callback):
    
    def __init__(self):
        self.train_loss = []
        self.test_loss = []
        
        self.fig_line = plt.figure(title='Training and Test Errors')
        self.lines = plt.plot([0], [0], colors=['DeepSkyBlue', 'Red'], labels=['Training', 'Test'],
                              display_legend=True)
        
        self.fig_wts = plt.figure(title='L1 Norm of Weights - by layer')
        self.bars = plt.bar([0],[0], colors=['MediumSeaGreen'], scales={'x': OrdinalScale()})
        self.train_label = Label(value='Train Accuracy: ', layout=Layout(width='800px'))
        self.test_label = Label(value='Test Accuracy: ', layout=Layout(width='800px'))
        self.prog_bar = FloatProgress(description='Epoch 0')
        self.layout = VBox([self.prog_bar, HBox([self.fig_line, self.fig_wts]), self.train_label, self.test_label],
                          layout=Layout(flex='1'))
        display(self.layout)
        
    def on_train_begin(self, logs={}):
        self.prog_bar.min, self.prog_bar.max = 0, self.params['nb_epoch']
        self.bars.x = list(np.arange(0, len(self.model.layers)))
        
    def on_epoch_end(self, epoch, log={}):
        epoch_train_acc = np.mean(np.argmax(self.model.predict(x_train_flat), axis=1)==y_train)
        epoch_test_acc = np.mean(np.argmax(self.model.predict(x_test_flat), axis=1)==y_test)
        
        self.train_loss.append(1 - epoch_train_acc)
        self.test_loss.append(1 - epoch_test_acc)

        self.lines.x, self.lines.y = np.arange(0, epoch), [self.train_loss, self.test_loss]
        
        weight_norm = []
        for layer in np.arange(0, len(self.model.layers)):
            wts = self.model.layers[layer].get_weights()[0]
            weight_norm.append(np.sum(np.abs(wts)) / wts.shape[0])
        self.bars.y = weight_norm
        
        self.prog_bar.description = 'Epoch ' + str(epoch + 1) + '/' + str(self.params['nb_epoch'])
        self.prog_bar.value += 1.
        
        self.train_label.value = 'Train Accuracy: ' + str(epoch_train_acc)
        self.test_label.value = 'Test Accuracy: ' + str(epoch_test_acc)

In [ ]:
x_train_flat = x_train.reshape(60000, 784)
x_test_flat = x_test.reshape(10000, 784)

In [ ]:
# convert class vectors to binary class matrices
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

In [ ]:
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(784,)))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.metrics.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'], verbose=False)

_ = model.fit(x_train_flat, y_train_cat,
          batch_size=batch_size,
          nb_epoch=epochs,
          verbose=False,
          validation_data=(x_test_flat, y_test_cat), callbacks=[Dashboard()])